In [32]:
import os
import openai
import json
import time
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat.chat_completion import ChatCompletion
from openai.types.beta import Assistant
from openai.types.beta.thread import Thread
from openai.types.beta.threads.run import Run

In [2]:
client : OpenAI = OpenAI()
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

In [3]:
from openai.types.chat.chat_completion import ChatCompletion
def chat_completion(prompt : str )-> str:
    response : ChatCompletion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            model="gpt-3.5-turbo-1106",
        )
    return response.choices[0].message.content

chat_completion("what is 1+1?")

'1+1 equals 2.'

In [ ]:

def chat_completion()-> str:
    completion : ChatCompletion = client.chat.completions.create(
        model = 'gpt-3.5-turbo-1106',
        messages = [
            {'role': 'system', 'content': 'You are a poetic assistant, skilled in explaining complex programming concepts in creative poetry'},
            {'role': 'user', 'content':'Compose a poem that explains the concept of recursion in programming'}
        ]
    )
    return completion.choices[0].message.content

print(chat_completion())

We can make the model return output in json format


In [ ]:
# even the output is in the json format, the type of output is string
# because the output of llm is always string
response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo-1106',
    response_format={'type': 'json_object'},
    messages=[
            {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
            {"role": "user", "content": "List of months that have 30 days"}
            ]
)

print(response.choices[0].message.content)
print(type(response.choices[0].message.content))

In [ ]:
# even if you dont specify response format, it still will be string
json_data = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo-1106',
    #response_format = {'type':'json_object'},
    messages = [
        {'role':'system', "content": "You are a helpful assistant designed to output JSON format"},
        {'role':'user', "content": "What are the advantages to convert the output of ChatGPT into JSON format"}
    ]
)

print(json_data.choices[0].message.content)
print(type(json_data.choices[0].message.content))

## Function Calling


In [ ]:
#custom function
import json

def get_current_weather(location: str, unit: str = 'fahrenheit')-> str:
    """Get the current weather in a given location"""
    if 'tokyo' in location.lower():
        return json.dumps({'location':'Tokyo', 
                        'temperature': '10',
                        'unit': 'celsius'})
    elif 'san francisco' in location.lower():
        return json.dumps({'location':'San francisco', 
                    'temperature': '72',
                    'unit': 'fahrenheit'})
    elif 'paris' in location.lower():
        return json.dumps({'location':'Paris', 
                        'temperature': '22',
                        'unit': 'celsius'})
    else:
        return json.dumps({'location':location, 
                        'temperature': 'unknown'})
        

In [ ]:
def run_conversation(main_request: str)->str:
    #Step 1: Send the conversation and available functions to the model
    
    messages = [{'role': 'user','content':main_request}] #user messages list
    
    #along with the prompt messages, we give details about the function
    
    tools=[#list of function to be passed along the prompt
        { #first function to be passed along the prompt
            'type':'function',# define what you are passing. Here it is function
            'function':{ #what the function is
                'name': 'get_current_weather', #name of the function
                'description': 'Get the current weather in a give location',
                #what the function is doing (important for NLP to understand 
                # what the function is used for)
                'parameters': { #what is passed as argument to function
                    'type': 'object',
                    'properties':{
                        'location': {#first parameter
                            'type': 'string', #type of location parameter
                            'description': 'The city and state e.g San Francisco, CA',
                            #description of the parameter
                        },
                        'unit': { #second parameter
                            'type':'string', #type of the unit parameter
                            'enum': ['celsius', 'fahrenheit']
                        }
                    },
                    'required': ['location'] #required parameter (if not 
                    #provided, the function will not run)
                }
            }
        }
    ]
    
    # First request
    
    response: openai.ChatCompletion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-1106", #model selection
        messages=messages, #message list
        tools=tools, #list of dictionary to be passed
        tool_choice="auto",  # automatically select function that is to be used
        #if general quesiton asked, it will respond with it's own knowledge
        #but if asked particular question, it will auto call function
    )
    
    response_message = response.choices[0].message
    display("* First Response: ", dict(response_message))

    
    tool_calls = response_message.tool_calls #additional parameter
    display("First Response Tool Calls: ", tool_calls) #give details about
    #list of function that is to be called
    
    #Step 2: Check if the model wanted to call a function
    
    if tool_calls: #if the list have any element then what to do
        #Step 3: Call the function. The JSON response may not always be correct
        #Be sure to handle errors
        
        available_functions = {
            "get_current_weather": get_current_weather,
        } #
        
        messages.append(response_message)  
        # extend conversation with assistant's reply
        
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name #extract the name of function
            
            function_to_call = available_functions[function_name] # check if the
            # present in available_functions. If so, assign the function name
            # to the variable
            
            function_args = json.loads(tool_call.function.arguments) #all the 
            #parameter of the function is converted into JSON (dictionary)
            
            function_response = function_to_call( 
                location=function_args.get("location"),#get location parameter
                unit=function_args.get("unit"), #get unit parameter
            )
            
            messages.append( #response after running custom function is appended
                            #to the messages list (thread)
                {
                    "tool_call_id": tool_call.id, #which function is running and
                    #its id
                    "role": "tool", #role of custom function (tool)
                    "name": function_name, #name of custom function 
                    "content": function_response, #response of the custom function
                }
            )  # extend conversation with function response
            
        display("* Second Request Messages: ", list(messages)) 
        
        second_response: openai.ChatCompletion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function 
        #response
        
        print("* Second Response: ", dict(second_response))
        
        return second_response.choices[0].message.content
    


In [ ]:
run_conversation("What's the weather like in San Francisco, Tokyo, and Paris?")

In [ ]:
def get_cryptocurrency_price(crypto: str)-> str:
    '''Get crypto current price'''
    if 'bitcoin' in crypto.lower():
        return json.dumps({'crypto': 'Bitcoin', 'price': '40,016'})
    elif 'ethereum'in crypto.lower():
        return json.dumps({'crypto': 'Ethereum', 'price': '2,222'})
    elif 'usdt' in crypto.lower():
        return json.dumps({'crypto': 'USDT', 'price': '0.999'})
    elif 'bnb' in crypto.lower():
        return json.dumps({'crypto': 'BNB', 'price': '290.33'})
    else:
        return json.dumps({'crypto': crypto, 'price':'unknown'})
    

In [ ]:
def get_price_conversation(main_request: str)->str:
    
    message_list = [{'role':'user', 'content':main_request}]
    tools=[
        {
            'type':'function',
            'function':{
                'name':'get_cryptocurrency_price',
                'description': 'Get current price of crypto currency',
                'parameters':{
                    'type': 'object',
                    'properties':{
                        'crypto':{
                            'type': 'string',
                            'description':'Name of the crypto currency to get current price'
                        }
                    },
                    'required': ['crypto']
                }
            }
        }
    ]
    
    
    response = openai.ChatCompletion.create(
        model = 'gpt-3.5-turbo-1106',
        messages = message_list,
        tools = tools,
        tool_choice = 'auto'
    )
    
    assistant_message = response.choices[0].message
    tool_calls = assistant_message.tool_calls
    
    if tool_calls:
        available_functions = {
            'get_cryptocurrency_price': get_cryptocurrency_price,
        }
        
        message_list.append(assistant_message)
        
        for tool_call in tool_calls:
            
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            
            function_response = function_to_call(
                crypto = function_args.get('crypto')
            )
            
            message_list.append({
                'tool_call_id': tool_call.id,
                'role':'tool',
                'name': function_name,
                'content': function_response
            })
        
        second_response = openai.ChatCompletion.create(
            model = 'gpt-3.5-turbo-1106',
            messages = message_list,
        )
        
        return second_response.choices[0].message.content

In [ ]:
get_price_conversation('What is the price of Ethereum today')

In [ ]:
get_price_conversation('What is the price of Bitcoin today')

In [ ]:
get_price_conversation('What is the price of PKR today')

In [ ]:
get_price_conversation('What the R&B music genre means?')

## OpenAI Assistant


In [ ]:
%pip install --upgrade openai

##### The first step is to create an assistant


In [ ]:
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

assistant: Assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-3.5-turbo-1106"
)

show_json(assistant)

##### The second step is to create a thread


In [ ]:
thread: Thread  = client.beta.threads.create()

##### The third step is to add the Messsage to thread

In threads, we have multiple messages between the user and the assistant. We have
to create messages, put them in a thread and then link them to the assistant
object
`thread` is independent of assistant


In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

show_json(message)

#we have not connected to an assistant, so assistant_id is none


### <b>Note even though you're no longer sending the entire history each time, you still will be charged for the tokens of the entire conversation</b>


##### The fourth step is to Run the Assistant

To get a completion from an Assistant for a given Thread, we must create a <b>Run</b>


In [ ]:
run: Run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)

show_json(run)

Creating `Run` is an asynchronous operation


In [ ]:
run: Run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

print(run)

In [ ]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

##### Next step is to check the Run status


In [ ]:
run = wait_on_run(run, thread)
show_json(run)

##### The next step is display the assistant response


In [ ]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

for m in reversed(messages.data):
    print(m.role + ": " + m.content[0].text.value)

In [11]:
assistant = client.beta.assistants.create(
    name = 'Math Tutor',
    instructions= 'You are a personal math tutor. Write and run code to answer math questions',
    tools = [{'type':'code_interpreter'}],
    model = 'gpt-3.5-turbo-1106'
)

In [12]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_2nwAuga7BbPvbxenlLG6hZRB', created_at=1706276816, metadata={}, object='thread')


In [13]:
message = client.beta.threads.messages.create( #we have to give three different parameters
    thread_id = thread.id, #thread id to which we want to link this message to
    role = 'user',
    content = 'Solve this problem: 3x + 11 = 14'
)

print(message)

ThreadMessage(id='msg_3DZp65mOVv4wwMrm0d6Vg4PR', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='Solve this problem: 3x + 11 = 14'), type='text')], created_at=1706276816, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_2nwAuga7BbPvbxenlLG6hZRB')


In [14]:
run = client.beta.threads.runs.create(
    #we have to pass two parameters
    #1) thread id 2) assistant id
    thread_id = thread.id,
    assistant_id= assistant.id
)

In [16]:
run = client.beta.threads.runs.retrieve(
    thread_id = thread.id,
    run_id = run.id,
)

messages = client.beta.threads.messages.list(
    thread_id = thread.id,
)

for message in reversed(messages.data): #we want to get the most oldest message
    #printed out first and then the most latest message from the assistatn
    print(message.role + ":" + message.content[0].text.value)

user:Solve this problem: 3x + 11 = 14
assistant:The solution to the equation 3x + 11 = 14 is x = 1.


#### Upload Files to AI


In [4]:
file = client.files.create(
    file=open("zia_profile.pdf", "rb"),
    purpose='assistants'
)

print(file)

FileObject(id='file-NY3NcOQlE8txFwnLZlxAgyZa', bytes=48802, created_at=1706420849, filename='zia_profile.pdf', object='file', purpose='assistants', status='processed', status_details=None)


`Create the assistant`


In [5]:
assistant: Assistant = client.beta.assistants.create(
    name="Student Support Assistant",
    instructions="You are a student support chatbot. Use your knowledge base to best respond to student queries about Zia U. Khan.",
    model="gpt-3.5-turbo-1106",
    tools=[{"type": "retrieval"}],
    file_ids=[file.id]
)

`Create a thread`


In [6]:
thread: Thread  = client.beta.threads.create()

print(thread)

Thread(id='thread_tpmRAgUdje3cUZp6KNibOSmK', created_at=1706420855, metadata={}, object='thread')


`Create a message`


In [7]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="When and which city Zia U. Khan was born? What was his first job?"
)

`Run the assistant`


In [8]:
run: Run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Pakistani. The user is the student of PIAIC."
)

`Display the assistant response`


In [9]:
run: Run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)



messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

for m in reversed(messages.data):
    print(m.role + ": " + m.content[0].text.value)

user: When and which city Zia U. Khan was born? What was his first job?


## Assistant Function Calling


`Creating function that is to be called`


In [11]:
import json

def show_json(message, obj):
    display(message, json.loads(obj.model_dump_json()))

In [23]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def getCurrentWeather(location:str, unit:str="fahrenheit")->str | dict | None:
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "los angeles" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})
    

def getNickname(location:str)->str:
    """Get the nickname of a city"""
    if "tokyo" in location.lower():
        return "tk"
    elif "los angeles" in location.lower():
        return "la"
    elif "paris" in location.lower():
        return "py"
    else:
        return location

`Creating an assistant`


In [24]:
assistant: Assistant = client.beta.assistants.create(
    instructions="You are a weather bot. Use the provided functions to answer questions.",
    model="gpt-3.5-turbo-1106",
    tools=[{
    "type": "function",
    "function": {
    "name": "getCurrentWeather",
    "description": "Get the weather in location",
        "parameters": {
            "type": "object",
            "properties": {
            "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
            "unit": {"type": "string", "enum": ["c", "f"]}
            },
            "required": ["location"]
            }
        }
    }, {
        "type": "function",
        "function": {
        "name": "getNickname",
        "description": "Get the nickname of a city",
        "parameters": {
            "type": "object",
            "properties": {
            "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
            },
            "required": ["location"]
        }
        } 
    }]
    )

`Create a thread`


In [25]:
from openai.types.beta.thread import Thread

thread: Thread  = client.beta.threads.create()

print(thread)

Thread(id='thread_Aql6QneoQTMw6CkXD28yu31v', created_at=1706421960, metadata={}, object='thread')


`Add a message to the thread`


In [26]:
from openai.types.beta.threads.thread_message import ThreadMessage

# First Request
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="How is the weather in Los Angles?"
)

In [27]:
dict(message)

{'id': 'msg_INaqT9wnIFaiJ8OeuBux38gn',
 'assistant_id': None,
 'content': [MessageContentText(text=Text(annotations=[], value='How is the weather in Los Angles?'), type='text')],
 'created_at': 1706421966,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_Aql6QneoQTMw6CkXD28yu31v'}

`Run the assistant`


In [28]:
from openai.types.beta.threads.run import Run

run: Run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

In [29]:
dict(run)

{'id': 'run_KmKK9lxhAk0MImJFiEPZEllX',
 'assistant_id': 'asst_Jms9baS8i0SGoZBl9WHH96Cl',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1706421987,
 'expires_at': 1706422587,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a weather bot. Use the provided functions to answer questions.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_Aql6QneoQTMw6CkXD28yu31v',
 'tools': [ToolAssistantToolsFunction(function=FunctionDefinition(name='getCurrentWeather', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['c', 'f']}}, 'required': ['location']}, description='Get the weather in location'), type='function'),
  ToolAssistantToolsFunction(function=FunctionDefinition(name='getNickname', parameters={'type': 'object

In [30]:
available_functions = {
    "getCurrentWeather": getCurrentWeather,
    "getNickname": getNickname
} 

In [33]:


  # Loop until the run completes or requires action
while True:
    runStatus = client.beta.threads.runs.retrieve(thread_id=thread.id,
                                                  run_id=run.id) #status of the run
    # Add run steps retrieval here for debuging
    run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id, run_id=run.id)
    # show_json("Run Steps:", run_steps)
    print(runStatus.status ,'.....') # print the status of run

    # This means run is making a function call   
    if runStatus.status == "requires_action":
        print(runStatus.status ,'.....')
        print("Status: ", "requires_action")
        show_json("submit_tool_outputs", runStatus.required_action)
        if runStatus.required_action.submit_tool_outputs and runStatus.required_action.submit_tool_outputs.tool_calls:
            print("toolCalls present:")
            toolCalls = runStatus.required_action.submit_tool_outputs.tool_calls

            tool_outputs = []
            for toolcall in toolCalls:
                function_name = toolcall.function.name
                function_args = json.loads(toolcall.function.arguments)
                
                if function_name in available_functions:
                    
                    
                    function_to_call = available_functions[function_name]
                    print(function_to_call,function_to_call.__name__=="getCurrentWeather","================================================================")
                  
                    if function_to_call.__name__ == "getCurrentWeather":
                        
                        response = function_to_call(
                        location=function_args.get("location"),
                        unit=function_args.get("unit")
                        )
                        
                        
                        tool_outputs.append({
                                  "tool_call_id": toolcall.id,
                                  "output": response
                              })
                    
                    elif function_to_call.__name__ == "getNickname":
                        response = function_to_call(
                          location=function_args.get("location")
                          )
                        tool_outputs.append({
                          "tool_call_id": toolcall.id,
                          "output": response,
                              })
            print(tool_outputs,">>>>>") 
            # Submit tool outputs and update the run
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs)
      
    elif runStatus.status == "completed":
        # List the messages to get the response
        print("completed...........logic")
        messages: list[ThreadMessage] = client.beta.threads.messages.list(thread_id=thread.id)
        for message in messages.data:
            role_label = "User" if message.role == "user" else "Assistant"
            message_content = message.content[0].text.value
            print(f"{role_label}: {message_content}\n")
        break  # Exit the loop after processing the completed run

    elif run.status == "failed":
      print("Run failed.")
      break

    elif run.status in ["in_progress", "queued"]:
      print(f"Run is {run.status}. Waiting...")
      time.sleep(5)  # Wait for 5 seconds before checking again

    else:
      print(f"Unexpected status: {run.status}")
      break

expired .....
Run is queued. Waiting...
expired .....
Run is queued. Waiting...
expired .....
Run is queued. Waiting...
expired .....
Run is queued. Waiting...
expired .....
Run is queued. Waiting...
expired .....
Run is queued. Waiting...
expired .....
Run is queued. Waiting...
expired .....
Run is queued. Waiting...
expired .....
Run is queued. Waiting...
expired .....
Run is queued. Waiting...
expired .....
Run is queued. Waiting...
expired .....
Run is queued. Waiting...
expired .....
Run is queued. Waiting...
expired .....
Run is queued. Waiting...
expired .....
Run is queued. Waiting...
expired .....
Run is queued. Waiting...
expired .....
Run is queued. Waiting...
expired .....
Run is queued. Waiting...
expired .....
Run is queued. Waiting...
expired .....
Run is queued. Waiting...
expired .....
Run is queued. Waiting...
expired .....
Run is queued. Waiting...
expired .....
Run is queued. Waiting...
expired .....
Run is queued. Waiting...
expired .....
Run is queued. Waiting...


KeyboardInterrupt: 